<a href="https://colab.research.google.com/github/aguinaldoabbj/Analyzing-Gender-IES/blob/master/4_geo_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geospatial Data Visualization 

## Introduction

Geospatial coordinates are very important when dealing with real world data. Geospatial data sets are mainly composed by latitude and longitude coordinates of the entities under study. So, a primary action would be to put these coordinates in a map.


## Our case study

In this stage we are going to stick with data from [Dados Abertos Natal](http://dados.natal.br/group/09132b97-d39f-4316-a859-240c31e98ef8?res_format=KML), but this time our target data will be a series of [KML (Keyhole Markup Language)](https://developers.google.com/kml/documentation) files available in this repository, regarding bus routes and stops.

We are going to start with the "Paradas" dataset, which refers to the mapped bus stops in the city. 

## Obtaing, Loading and Preparing Data

We can use 'wget' again to download data to our workspace in Colab.

In [1]:
!wget -c https://raw.githubusercontent.com/aguinaldoabbj/minicourse_open_data_natal_2019/master/data/paradas-unificadas.kml

--2019-03-19 19:35:38--  https://raw.githubusercontent.com/aguinaldoabbj/minicourse_open_data_natal_2019/master/data/paradas-unificadas.kml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1444962 (1.4M) [text/plain]
Saving to: ‘paradas-unificadas.kml’

paradas-unificadas. 100%[===================>]   1.38M  --.-KB/s    in 0.05s   

2019-03-19 19:35:38 (25.5 MB/s) - ‘paradas-unificadas.kml’ saved [1444962/1444962]



We have to load two new libraries to deal with KML data: fastKML and BeatifulSoup. The former is to decode the KML and the latter is to parse it. As they are not native in Colab, we have to download and install them with 'pip' utility.

In [2]:
!pip install fastkml
!pip install bs4

    100% |████████████████████████████████| 71kB 4.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/55/01/ea/6191eb73e0894743d02b33a2b1a570e85242844d810804fbf2
  Stored in directory: /root/.cache/pip/wheels/60/6e/a7/3d3eef59ac84a86663d0f5c5a92091f5056e9aeb6588c4de34
Successfully built fastkml pygeoif


Now we can load the installed libraries.

In [0]:
from fastkml import kml
from bs4 import BeautifulSoup


KML files are very similar to XML files, so that we need a specific dealing strategy. We can load the KML file into Python by using Python's native *open()* statement. 

In [0]:
#Read file into string and convert to UTF-8
with open("paradas-unificadas.kml", 'rt', encoding="utf-8") as myfile:
     kml_doc=myfile.read()

And use fastKML to decode our KML file now stored in variable *kml_doc*.

In [0]:
#start fast kml engine
k = kml.KML()
# Read in the KML string
k.from_string(kml_doc)

By browsing KML features (using fastKML's *features()* method), it is possible to locate where data is:

In [6]:
# browsing features's levels
l1 = list(k.features())
print(len(l1))

1


In [7]:
l2 = list(l1[0].features())
print(len(l2))

1


In [8]:
l3 = list(l2[0].features())
print(len(l3))

1958


In [9]:
print(l3[0].to_string()) #getting data from the first placemark

<kml:Placemark xmlns:kml="http://www.opengis.net/kml/2.2">
  <kml:name>Parada - 16 de Dezembro - O088</kml:name>
  <kml:description>&gt; Linhas de ônibus:&lt;br&gt;* 59 - Guarapes/Brasília Teimosa, via Bom Pastor;&lt;br&gt;* 599 - Guarapes/Mirassol, via Av. Cap.-Mor Gouveia;</kml:description>
  <kml:visibility>1</kml:visibility>
  <kml:Style>
    <kml:IconStyle>
      <kml:scale>0.9</kml:scale>
      <kml:Icon>
        <kml:href>http://maps.google.com/mapfiles/kml/pushpin/blue-pushpin.png</kml:href>
      </kml:Icon>
    </kml:IconStyle>
  </kml:Style>
  <kml:Point>
    <kml:coordinates>-35.278631,-5.833069,0.000000</kml:coordinates>
  </kml:Point>
</kml:Placemark>



Gotcha! Data is in the hierarchy level 3 in the KML. It seems we have 1958 placemarks. Now we can try to parse this XML-like data with BeautifulSoup, a very powerful library which can be used to parse several types of markup data.

In [0]:
# Use bs4 xml parser to parse kml data for each placemark
bus_stop_coords = [] #an array to store the retrieved coordinates
bus_stop_names = []
for p in l3:
    soup = BeautifulSoup(p.to_string(), "lxml-xml") #getting each placemark
    coord = soup.find('kml:coordinates') #searching coodinates
    coord = coord.get_text().split(",")[0:2] #getting only lat/long
    name = soup.find('kml:name') #searching names
    name = name.get_text()
    bus_stop_coords.append(coord) #appeding coordinates to array
    bus_stop_names.append(name)

The previous stage resulted in two arrays, *bus_stop_coords* and *bus_stop_names*, corresponding to bus stops coordinates and names, respectively. Based on these arrays, it is possible to build a Pandas dataframe to make data manipulation easier.

In [11]:
#loading pandas
import pandas as pd

#building new dataframe
bus_stops_df = pd.DataFrame({'Name':bus_stop_names,'Coordinate':bus_stop_coords})# arrays are mapped into dataframe columns

#checking if things are OK
bus_stops_df.head()

,Coordinate,Name
0,"[-35.278631, -5.833069]",Parada - 16 de Dezembro - O088
1,"[-35.246364, -5.843083]",Parada - 17 de Dezembro - O068
2,"[-35.280042, -5.838146]",Parada - 28 de Fevereiro -O082
3,"[-35.182530, -5.799994]",Parada - L001
4,"[-35.182753, -5.799874]",Parada - L002


In [12]:
bus_stops_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1958 entries, 0 to 1957
Data columns (total 2 columns):
Coordinate    1958 non-null object
Name          1958 non-null object
dtypes: object(2)
memory usage: 30.7+ KB


## Spatial visualization and Analysis

Now that we have a brand new dataframe contaning a bunch of bus stop coordinates, a next step would be to plot them in a map. To do so, I recommend a very promising library for spatial visualizations and analysis called *folium*. Let's get the latest *folium* with pip and load it:

In [13]:
!pip install folium==0.8.3
import folium

    100% |████████████████████████████████| 92kB 5.6MB/s 
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: folium 0.2.1
    Uninstalling folium-0.2.1:
      Successfully uninstalled folium-0.2.1


Once folium is loaded, we can instantiate a simple map of our city:

In [14]:
# building a map with folium
natal = folium.Map(
    location=[-5.799214, -35.237978], #a central coordinate to start zooming the map
    width=400,height=600,
    zoom_start=12, # zoom: the higher the zoom, the closer it gets to ground
    tiles='Stamen Terrain' #map types
)

natal #show the map

## Plotting Coordinates in a Map

A next stage is to plot some of the bus stop coordinates in the map by iterating over dataframe rows with the *iterrows()* method. Don't try to plot all coordinates!


In [15]:
for nrow, row in bus_stops_df.sample(50).iterrows(): #iterating over dataframe rows
  lat=float(row.Coordinate[1]) #getting latitude from coodinate (str -> float)
  long=float(row.Coordinate[0]) #getting longitude from coodinate (str -> float)
  folium.Marker(
        location=[lat,long],
        popup=row.Name
               ).add_to(natal)
    
    
natal

### Exercise

![alt text](https://cdn.dribbble.com/users/2344801/screenshots/4774578/alphatestersanimation2.gif =150x120)

Use the same mechanism developed so far to plot the coordinates of the routes taken by bus line [65](http://dados.natal.br/dataset/6727af4e-868a-42d8-98b6-db5f2a717293/resource/b8693adb-bbf3-4863-909d-8683b3353114/download/65.kml). I recommend you to create a new notebook in Colab to do this exercise. Head over to Colab and click on New Notebook (choose Python 3) to create a new notebook. By default it’s created in your Google Drive under Colab Notebooks folder. Since it’s in Google Drive, you can move it around and share it just like any other Drive documents.



In [0]:
# put your code in this cell

## Adding Layers



Folium has an extra feature that permits layers to be passed to the map as an overlay, and multiple layers can be visualized on the same map. A suitable layer in the scope of our study would refer to the neighborhoods that compose the city under study. To build such a layer we need to know the coordinates of neighboors' boundaries. [Geojson](https://tools.ietf.org/html/rfc7946) are a common source of this set of coordinates.

Geojson is an open standard format designed for representing simple geographical features. A Geojson file describing Natal in terms of its neighborhoods can be found in this project:


Let's use 'wget' utility again to download the Geojson file regarding our city:

In [16]:
!wget -c https://raw.githubusercontent.com/aguinaldoabbj/minicourse_open_data_natal_2019/master/data/natal.geojson

--2019-03-19 20:01:46--  https://raw.githubusercontent.com/aguinaldoabbj/minicourse_open_data_natal_2019/master/data/natal.geojson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 296067 (289K) [text/plain]
Saving to: ‘natal.geojson’

natal.geojson       100%[===================>] 289.13K  --.-KB/s    in 0.03s   

2019-03-19 20:01:46 (9.58 MB/s) - ‘natal.geojson’ saved [296067/296067]



Now we can load it in Python by using library *json*

In [32]:
import json
# import geojson file and load it with json
# load the data and use 'UTF-8'encoding

with open("natal.geojson", encoding='UTF-8') as file:
    geo_json_natal = json.load(file)
    
    
print(json.dumps(geo_json_natal, indent=2)) #pretty printing json file.

{
  "type": "FeatureCollection",
  "generator": "overpass-ide",
  "copyright": "The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.",
  "timestamp": "2017-10-09T00:39:02Z",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "@id": "relation/388146",
        "admin_level": "10",
        "boundary": "administrative",
        "is_in": "Natal",
        "name": "Pitimbu",
        "place": "suburb",
        "type": "boundary"
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -35.2251535,
              -5.8800875
            ],
            [
              -35.2245789,
              -5.8789859
            ],
            [
              -35.2235407,
              -5.8773961
            ],
            [
              -35.2216713,
              -5.8748329
            ],
            [
              -35.219967,
              -5.8725269
            ],


As you can see, this Geojson describes a polygons for each neighborhood. Now that we have *geo_json_natal* object, we can use folium's GeoJson feature to build a layer over the original map we have built previously.

In [37]:
# rebuilding Natal Map to get rid of markers
natal = folium.Map(
    location=[-5.799214, -35.237978], #a central coordinate to start zooming the map
    width=400,height=600,
    zoom_start=12, # zoom: the higher the zoom, the closer it gets to ground
    tiles='Stamen Terrain' #map types
)

folium.GeoJson(
    geo_json_natal,
    name='Neighborhoods in Natal'
    ).add_to(natal)

natal

Knowing only neighborhoods' boundaries does not help much in visualization. Once the Geojson file also contains the names for each neighborhood, we can also put them in our neighborhood layer, however, to do so we have to get deeper into Geojson and browse its structure. More on parsing json files in Python can be found [here](https://docs.python.org/3/library/json.html).

A stage in this process is to build in-memory polygons regarding neighborhoods' boundaries (browsing the loaded Geojson object) and name them. We can use library ***shapely*** to build a [dictionary](https://www.w3schools.com/python/python_dictionaries.asp) of polygons and their respective names.

In [44]:
from shapely.geometry import Polygon

poly_dict = {} #dictionary
# list all neighborhoods and build a dictionary.
for neigh in geo_json_natal['features']: #browsing Geojson features (each feature is a neighborhood)
        #build a polygon from boundaries coordinates and put it in the dictionary
        poly_dict[neigh['properties']['name']] = Polygon(neigh['geometry']['coordinates'][0])
      
#we can see the dictionary
poly_dict

{'Alecrim': <shapely.geometry.polygon.Polygon at 0x7f75b4d2bac8>,
 'Areia Preta': <shapely.geometry.polygon.Polygon at 0x7f75b4d2b828>,
 'Barro Vermelho': <shapely.geometry.polygon.Polygon at 0x7f75b4d2b198>,
 'Bom Pastor': <shapely.geometry.polygon.Polygon at 0x7f75b4d2bb00>,
 'Candelária': <shapely.geometry.polygon.Polygon at 0x7f75b4d2b1d0>,
 'Capim Macio': <shapely.geometry.polygon.Polygon at 0x7f75b4d2b0b8>,
 'Cidade Alta': <shapely.geometry.polygon.Polygon at 0x7f75b4d2ba58>,
 'Cidade Nova': <shapely.geometry.polygon.Polygon at 0x7f75b4d2b898>,
 'Cidade da Esperança': <shapely.geometry.polygon.Polygon at 0x7f75b4d2b8d0>,
 'Dix-Sept Rosado': <shapely.geometry.polygon.Polygon at 0x7f75b4d2bc18>,
 'Felipe Camarão': <shapely.geometry.polygon.Polygon at 0x7f75b4d2b9e8>,
 'Guarapes': <shapely.geometry.polygon.Polygon at 0x7f75b4d2ba20>,
 'Igapó': <shapely.geometry.polygon.Polygon at 0x7f75b4d2b5f8>,
 'Lagoa Azul': <shapely.geometry.polygon.Polygon at 0x7f75b4d2b0f0>,
 'Lagoa Nova': <sh

This dictionary makes easy the process of naming the regions described in Geojson. We are going to put the names as markers located at the centroid of the neighborhood.

In [49]:
from folium.features import DivIcon #need this to put names instead of traditional markers

# Addin markers for neighborhoods
for neigh in list(poly_dict): #list dictionary keys
        name = neigh
        x,y = poly_dict[neigh].centroid.coords.xy
        lat = y[0]
        log = x[0]
        #Draw a small circle
        html_str='<div style="font-size: 6pt"><b>'+neigh+'</b></div>'
        folium.map.Marker([lat,log],
                        icon=DivIcon(
                        icon_size=(20,36),
                        icon_anchor=(0,0),
                        html=html_str,
                        )).add_to(natal)     
        
natal